In [2]:
from bs4 import BeautifulSoup
import requests
import dotenv
import re
import pandas as pd
import os
import json
import requests
from datetime import date
import datetime
import sqlalchemy # install if needed
import pymysql
import dotenv
import pytz

In [6]:
# here, I manually define the cities of interest with a unique key. I can also call them from mySQL
cities = {"BER":"Berlin", "BCN":"Barcelona", "STR":"Stuttgart", "LPZ":"Leipzig"}

#sql_query = pd.read_sql('SELECT city_id, city FROM city', con)
#cities = dict(zip(sql_query.city_id, sql_query.city))

{'BER': 'Berlin', 'BCN': 'Barcelona', 'STR': 'Stuttgart', 'LPZ': 'Leipzig'}

In [7]:
# City Wiki Function

def scrap_wiki(cities):

    city = []
    city_id = []
    country = []
    population = []
    lat = []
    lon = []
    today = []
    
    schema="schema"
    host="AWS_host"
    dotenv.load_dotenv() # the api key is in a .env file in the same directory
    password = os.getenv('sql_password')
    user="admin"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    

    for key, c in cities.items():
        url = f"https://en.wikipedia.org/wiki/{c}"
        # 3. download html with a get request
        headers = {'Accept-Language': 'en-US,en;q=0.8'}
        response = requests.get(url, headers = headers)
        print(c, response.status_code) # 200 status code means OK! check
        soup = BeautifulSoup(response.content, "html.parser")
        city.append(c)
        country.append(soup.select("td.infobox-data")[0].get_text())
        if soup.select("th.infobox-header:-soup-contains('Population')"):
            population.append(soup.select("th.infobox-header:-soup-contains('Population')")[0].parent.find_next_sibling().contents[1].get_text())
        lat.append(soup.select(".latitude")[0].get_text())
        lon.append(soup.select(".longitude")[0].get_text())
        today.append(date.today())
        city_id.append(key)
        # elevation = 
    wiki_df = pd.DataFrame({"city_id":city_id,
                "city":city, 
                 "country":country,
                "population":population,
                 "lat":lat, 
                 "lon":lon,
                    "timestamp": today,
                    "city_id":city_id})
    wiki_df["lat"] = wiki_df["lat"].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
    wiki_df["lon"] = wiki_df["lon"].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
    wiki_df["population"] = wiki_df["population"].str.replace(',', '', regex=False)
    # saving it as a csv
    #wiki_df.to_csv("city_wiki.csv", index=False)
    wiki_df.to_sql('city', 
              if_exists='append', 
              con=con, 
              index=False)
    return(wiki_df)

In [9]:
wiki_df = scrap_wiki(cities)
wiki_df

Berlin 200
Barcelona 200
Stuttgart 200
Leipzig 200


,city_id,city,country,population,lat,lon,timestamp
0,BER,Berlin,Germany,3850809,52.3112,13.2418,2023-08-02
1,BCN,Barcelona,Spain,1620343,41.2258,02.1037,2023-08-02
2,STR,Stuttgart,Germany,626275,48.4639,09.1048,2023-08-02
3,LPZ,Leipzig,Germany,601866,51.2024,12.2230,2023-08-02


In [10]:
# ICAO Function  
def airports_per_city(wiki_df):
    
    schema="schema"
    host="AWS_host"
    dotenv.load_dotenv() # the api key is in a .env file in the same directory
    password = os.getenv('sql_password')
    user="admin"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    

    lat = wiki_df["lat"]
    long = wiki_df["lon"]
    city_id = wiki_df["city_id"]
    list = []
    airport_df = []

    for lat, long, city_id in zip(lat, long, city_id):

        url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
    
        querystring = {"lat":lat,"lon":long,"radiusKm":"100","limit":"10"}

        headers = {
        "X-RapidAPI-Key": "0b7cffd425mshbf932b1b5f7e633p187a96jsna5aedb4ce276",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
    
        response = requests.get(url, headers=headers, params=querystring)
        print(city_id, response)
        df = pd.DataFrame(pd.json_normalize(response.json()['items']))
        df["city_id"] = city_id
        list.append(df)
    airport_df = pd.concat(list)
    airport_df = airport_df[["city_id", "icao", "name", "countryCode", "location.lat", "location.lon"]]
    airport_df.rename(columns={"countryCode":"country_code", "location.lat":"lat", "location.lon":"lon"}, inplace=True)
    airport_df.to_sql('airport', 
              if_exists='append', 
              con=con, 
              index=False)  
    return (airport_df)

In [ ]:
airport_df = airports_per_city(wiki_df)
airport_df

BER <Response [200]>
BCN <Response [200]>
STR <Response [200]>
LPZ <Response [200]>
